# Preprocessing Input Images of Marine Plastic
This notebook summarises the steps performed to preprocess the marine plastic images.
The dataset was sourced from [DeepData](https://drive.google.com/drive/folders/1fsS_u2QpbRGynYkP6-D6cfvq8r0hpjXI).

Boundary boxes to locate objects within the figures were determined using YoloV5. The images, all containing marine plastic, can be found in `Images` and the corresponding boundary boxes in `Labels`.

We preprocess the datasets to deal with the different figure sizes and the fact that the marine plastic and jelly fish datasets come from different sources. Firstly, a symmetric padding layer is added to make the figure sizes square. Secondly, the pixel numbers are rescaled to 100 x 100.


In [30]:
# Importing packages, pip install missing packages

import os
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from tqdm import tqdm



In [31]:
# Define auxilary functions

def read_label_file(label_file_path, img_width, img_height):
    boxes = []
    with open(label_file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            xmin = int((x_center - width / 2) * img_width)
            xmax = int((x_center + width / 2) * img_width)
            ymin = int((y_center - height / 2) * img_height)
            ymax = int((y_center + height / 2) * img_height)
            boxes.append((xmin, ymin, xmax, ymax))
    return boxes

def crop_figure(image_path, label_file_path):
    img = Image.open(image_path)
    img_array = np.array(img)
    img_width, img_height = img.size
    boxes = read_label_file(label_file_path, img_width, img_height)

    cropped_images = []
    for box in boxes:
        xmin, ymin, xmax, ymax = box
        cropped_images.append(img_array[ymin:ymax, xmin:xmax])
    return cropped_images

def rescale_image(image_array, new_shape=(100, 100)):
    rescaled_image = resize(image_array, (new_shape[0], new_shape[1], image_array.shape[2]), anti_aliasing=True)
    rescaled_image = (rescaled_image * 255).astype(np.uint8)
    return rescaled_image

def add_padding(img):
    height, width, channels = img.shape
    
    # Calculate padding needed to make the image square
    pad_vertical = abs(height - width) // 2
    pad_horizontal = abs(height - width) - pad_vertical

    if height > width:
        return np.pad(img, ((0, 0), (pad_vertical, pad_horizontal), (0, 0)), 'constant', constant_values=0)
    else:
        return np.pad(img, ((pad_vertical, pad_horizontal), (0, 0), (0, 0)), 'constant', constant_values=0)

def plot_figures(img,img_padded,img_rescaled):
    fig, axes = plt.subplots(1, 3)
    
    axes[0].imshow(img)
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(img_padded)
    axes[1].set_title('Padded Image')
    axes[1].axis('off')
    
    axes[2].imshow(img_rescaled)
    axes[2].set_title('Rescaled Image')
    axes[2].axis('off')
            

In [33]:
# Preprocess all images

all_files = os.listdir('Images/')
filenames = sorted([f.split('.jpg')[0] for f in all_files if f.endswith('.jpg')])

plastic_images = []

for fn in tqdm(filenames): #filenames[:50]
    img_path = f'Images/{fn}.jpg'
    label_path = f'Labels/{fn}.txt'
    
    cropped_images = crop_figure(img_path,label_path)

     # Loop through the boundary boxes
    for img in cropped_images:
        width, height, rgb = img.shape
        if width < 50 or height< 50: continue

        img_padded = add_padding(img)
        img_rescaled = rescale_image(img_padded)
        # plot_figures(img,img_padded,img_rescaled) # To plot all image preprocessing; e.g. loop through filenames[:10] 
        plastic_images.append(img_rescaled)

# Save data
np.save('plastic_images.npy', plastic_images)

labels = ['plastic']*len(plastic_images)
np.save('labels.npy', labels)


100%|██████████| 624/624 [00:07<00:00, 79.47it/s] 
